In [2]:
# download pre built models.
%%python -m spacy download en_core_web_sm
%%python -m nltk.downloader wordnet


✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')


In [6]:
import sys
import nltk

from api import UbiVocab

data_dir = "../../data/"
ubi_vocab = UbiVocab(data_dir = data_dir)
article = (f"This is an article about the damaging effects of not learning vocabulary. "
                f" This program seeks to create a routine exercise for learning vocabulary over time.")

# Process the input article above.
ubi_vocab.process_article(article = article, num_sentences = 1)


Reading existing file at ../../data/gre_vocab.csv
2021-04-27 02:41:48,754 - main - INFO - Loading in spacy and sentence transformer models.
2021-04-27 02:41:49,431 - transformer - INFO - Reading in pretrained model bert-base-nli-mean-tokens
2021-04-27 02:41:49,431 - sentence_transformers.SentenceTransformer - INFO - Load pretrained SentenceTransformer: bert-base-nli-mean-tokens
2021-04-27 02:41:49,432 - sentence_transformers.SentenceTransformer - INFO - Did not find folder bert-base-nli-mean-tokens
2021-04-27 02:41:49,432 - sentence_transformers.SentenceTransformer - INFO - Try to download model from server: https://sbert.net/models/bert-base-nli-mean-tokens.zip
2021-04-27 02:41:49,434 - sentence_transformers.SentenceTransformer - INFO - Load SentenceTransformer from folder: /Users/kevinzen/.cache/torch/sentence_transformers/sbert.net_models_bert-base-nli-mean-tokens
2021-04-27 02:41:51,186 - sentence_transformers.SentenceTransformer - INFO - Use pytorch device: cpu
2021-04-27 02:41:51

0    This is an article about the detrimental effects of not learning vocabulary.  This program seeks to create a quotidian exercise for learning vocabulary over time.
dtype: object

In [7]:
# View the new article with replaced words.
ubi_vocab.new_article

0    This is an article about the detrimental effects of not learning vocabulary.  This program seeks to create a quotidian exercise for learning vocabulary over time.
dtype: object

In [8]:
# View the underlying reasons for the replacement.
ubi_vocab.highlights_df

,word,syn,orig_text,mod_text,sim_score,syn_pos_context,syn_pos,synset,lesk
0,detrimental,damaging,This is an article about the damaging effects of not learning vocabulary,This is an article about the detrimental effects of not learning vocabulary,0.983425,adjective,adjective,Synset('damaging.s.01'),Synset('damaging.s.01')
1,mundane,routine,This program seeks to create a routine exercise for learning vocabulary over time,This program seeks to create a mundane exercise for learning vocabulary over time,0.787758,adjective,adjective,Synset('everyday.s.01'),Synset('mundane.s.03')
2,quotidian,routine,This program seeks to create a routine exercise for learning vocabulary over time,This program seeks to create a quotidian exercise for learning vocabulary over time,0.930106,adjective,adjective,Synset('everyday.s.01'),Synset('everyday.s.01')
